In [8]:
import tensorflow as tf
import pandas as pd
import math

In [2]:
sess = tf.InteractiveSession()

# Import Data (MNIST)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
dataset_address = "./mnist/"
mnist = input_data.read_data_sets(dataset_address, one_hot = True)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [5]:
print ((mnist.train.images[0,:]).size)
print (mnist.train.images.shape)
print (mnist.train.labels.shape)

784
(55000, 784)
(55000, 10)


# Design The Computation Graph

In [53]:
# Default Constants

Number_Classes = 10
batch_size = 100
learning_rate = 0.12
Image_Pixels = (mnist.train.images[0,:]).size
hidden_1_neurons = 15
hidden_2_neurons = 8
hidden_3_neurons = 0
outLayer_neurons = Number_Classes
images_placeholder = tf.placeholder(tf.float32, shape=(None, Image_Pixels))
labels_placeholder = tf.placeholder(tf.float32, shape=(batch_size))


# Build the graph

# Hidden 1
with tf.name_scope('hidden_1'):
    weights_1 = tf.Variable(tf.truncated_normal([Image_Pixels, hidden_1_neurons],
                                               stddev=1.0/math.sqrt(float(Image_Pixels))), name='weights')
    biases_1 = tf.Variable(tf.zeros([hidden_1_neurons]), name='biases')
    hidden_1_out = tf.nn.relu(tf.matmul(images_placeholder, weights_1) + biases_1)
# Hidden 2
with tf.name_scope('hidden_2'):
    weights_2 = tf.Variable(tf.truncated_normal([hidden_1_neurons, hidden_2_neurons],
                                               stddev=1.0/math.sqrt(float(hidden_1_neurons))), name='weights')
    biases_2 = tf.Variable(tf.zeros([hidden_2_neurons]), name='biases')
    hidden_2_out = tf.nn.relu(tf.matmul(hidden_1_out, weights_2) + biases_2)
# Output Layer
with tf.name_scope('out'):
    weights_out = tf.Variable(tf.truncated_normal([hidden_2_neurons, Number_Classes],
                                               stddev=1.0/math.sqrt(float(hidden_2_neurons))), name='weights')
    biases_out = tf.Variable(tf.zeros([Number_Classes]), name='biases')
    outLayer_output = tf.matmul(hidden_2_out, weights_out) + biases_out

# Training The NNET

- Before Variables can be used within a session, they must be initialized using that session. This step takes the initial values (in this case tensors full of zeros) that have already been specified, and assigns them to each Variable. This can be done for all Variables at once:
    - sess.run(tf.global_variables_initializer())
- We can specify a loss function just as easily. Loss indicates how bad the model's prediction was on a single example; we try to minimize that while training across all the examples. Here, our loss function is the cross-entropy between the target and the softmax activation function applied to the model's prediction. As in the beginners tutorial, we use the stable formulation:
    - One very common, very nice function to determine the loss of a model is called "cross-entropy."
    - To implement cross-entropy we need to first add a new placeholder to input the correct answers: expected_output = tf.placeholder(tf.float32, shape=[None, Number_Classes]) 
    - cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=expected_output, logits=outLayer_output))
    - Note that tf.nn.softmax_cross_entropy_with_logits internally applies the softmax on the model's unnormalized model prediction and sums across all classes, and tf.reduce_mean takes the average over these sums.
- Because TensorFlow knows the entire graph of your computations, it can automatically use the backpropagation algorithm to efficiently determine how your variables affect the loss you ask it to minimize. Then it can apply your choice of optimization algorithm to modify the variables and reduce the loss:
    - train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
- Training iteration
- We load 100 training examples in each training iteration. We then run the train_step operation, using feed_dict to replace the placeholder tensors x and y_ with the training examples. Note that you can replace any tensor in your computation graph using feed_dict -- it's not restricted to just placeholders.

In [54]:
sess.run(tf.global_variables_initializer())

expected_output = tf.placeholder(tf.float32, shape=[None, Number_Classes])

# loss function
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=expected_output, logits=outLayer_output))

# train steps and method
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# training iteration
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={images_placeholder: batch[0], expected_output: batch[1]})

# evaluation and testing
correct_prediction = tf.equal(tf.argmax(outLayer_output, 1), tf.argmax(expected_output, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={images_placeholder: mnist.test.images, expected_output: mnist.test.labels}))

0.9303
